## 1. 데이터 불러오기

'1_훼손 영역에 대한 검출 가능성.ipynb'에서 생성한 데이터를 사용.

In [1]:
import os

DATA_PATH = "./data/"
DATA_LIST_PATH = f"{DATA_PATH}.datalist"

# --------------------------------

# 1. 불러와야 할 데이터들의 파일명 수집
if not os.path.exists(DATA_PATH): raise Exception("No data.")

file_basename_list = []

with open(DATA_LIST_PATH, "r") as f:
    file_basename_list = f.readlines()
    file_basename_list = [file_basename.rstrip() for file_basename in file_basename_list]

# --------------------------------

# 2. 데이터를 불러오는 함수
def load(file_basename_list=file_basename_list):
    for file_basename in file_basename_list:
        file_name, file_extension = os.path.splitext(file_basename)

        raw  = cv2.imread(f"{DATA_PATH}{file_basename}", cv2.IMREAD_COLOR)
        mask = cv2.imread(f"{DATA_PATH}{file_name}_mask{file_extension}", cv2.IMREAD_GRAYSCALE)

        if raw is None:
            raise Exception(f"Imread failed: {file_basename} from {file_basename_list}")

        yield raw, mask

## 2. 선호/비선호 - 정상/비정상 영역 판단

다음과 같이 레이블을 부여한다.

* 비선호(정상): 0
* 선호(훼손): 1


In [3]:
import cv2

file_path = 'partial_01.JPG'

MIN_CONT_AREA = 20

runThisCell = True

# --------------------------------

if runThisCell:
    mx, my = 0, 0 # 마우스 포인터 좌표
    win_name = 'create dataset'
    idx = 0

    # --------------------------------

    file_name, file_extension = os.path.splitext(file_path)
    frame_raw, frame_mask = load([file_path]).__next__()

    # --------------------------------

    def mousePosition(event,x,y,flags,param):
        """마우스 포인터의 좌표 갱신"""
        global mx, my
        if event == cv2.EVENT_MOUSEMOVE:
            mx, my = x, y

    def makePoints(n, cx, cy):
        """중심의 좌표(cx,cy)와 한 변의 길이(n)을 이용하여 정사각형의 두 꼭짓점의 좌표를 계산."""
        x0, y0 = round(cx - n/2), round(cy - n/2)
        x1, y1 = round(cx + n/2), round(cy + n/2)
        return x0, y0, x1, y1

    def drawSqure(frame, n, cx, cy):
        """주어진 중심의 좌표(cx,cy)와 한 변의 길이(n)을 이용하여 프레임상에 정사각형을 그리고,
        그 프레임과 그려진 정사각형의 두 꼭짓점 좌표를 반환.\n
        이 함수에서는, 프레임 밖으로 벗어나는 정사각형에 대한 예외처리를 하였다."""
        h, w = frame.shape[:2]
        dx, dy = 0, 0

        x0, y0, x1, y1 = makePoints(n, cx, cy)

        # 예외처리 시작
        if (x0 < 0): dx = -x0
        if (x1 > w): dx = -x1 + w            cv2.imwrite(file_saveto_path, frame)ㄷ다.

        if (y0 < 0): dy = -y0
        if (y1 > w): dy = -y1 + h
        
        # 예외처리 적용
        x0, y0, x1, y1 = makePoints(n, cx+dx, cy+dy)

        cv2.rectangle(frame, (x0,y0), (x1,y1), (0,0,255), 1)
        return frame, x0, y0, x1, y1

    def saveLabeledFrame(frame, label):
        f"""데이터를 저장할 경로: '{DATA_PATH}'에 주어진 레이블 명과 동일한 이름의 디렉토리에 주어진 프레임을 저장한다."""
        class_path = os.path.join(DATA_PATH, label)
        new_file_name = f"{file_name}_{idx}{file_extension}"

        if not os.path.exists(class_path): os.mkdir(class_path)
        cv2.imwrite(os.path.join(class_path, new_file_name), frame)
        idx += 1

    # --------------------------------

    cv2.namedWindow(win_name)
    cv2.setMouseCallback(win_name, mousePosition)

    # --------------------------------

    for cont in cv2.findContours(frame_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[-2]:
        if cv2.contourArea(cont) >= MIN_CONT_AREA:
            cv2.drawContours(frame_mask, [cont], -1, (0,0,255), -1)
        else:
            cv2.drawContours(frame_mask, [cont], -1, (0,128,128), -1)
        
    frame_addWeighted = cv2.addWeighted(frame_raw, 0.75, frame_mask, 0.75, 0)

    # --------------------------------
    
    while True:
        frame, x0, y0, x1, y1 = drawSqure(frame_addWeighted.copy(), 28, mx, my)
        cv2.imshow(win_name, frame)

        key = cv2.waitKey(10)
        if key == ord('q'):
            break
        if key in [ord(x) for x in "0123456789"]:
            label = chr(key)
            saveLabeledFrame(frame_raw[y0:y1, x0:x1], label)

    # --------------------------------

    cv2.destroyWindow(win_name)
    print('Done.')